In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Amyotrophic_Lateral_Sclerosis"
cohort = "GSE26927"

# Input paths
in_trait_dir = "../../input/GEO/Amyotrophic_Lateral_Sclerosis"
in_cohort_dir = "../../input/GEO/Amyotrophic_Lateral_Sclerosis/GSE26927"

# Output paths
out_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/GSE26927.csv"
out_gene_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE26927.csv"
out_clinical_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/clinical_data/GSE26927.csv"
json_path = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Common neuroinflammatory pathways in neurodegenerative diseases."
!Series_summary	"Neurodegenerative diseases of the central nervous system are characterised by pathogenetic cellular and molecular changes in specific areas of the brain that lead to the dysfunction and/or loss of explicit neuronal populations. Despite exhibiting different clinical profiles and selective neuronal loss, common features such as abnormal protein deposition, dysfunctional cellular transport, mitochondrial deficits, glutamate excitotoxicity and inflammation are observed in most, if not all, neurodegenerative disorders, suggesting converging pathways of neurodegeneration. We have generated comparative genome-wide gene expression data for Alzheimer’s disease, amyotrophic lateral sclerosis, Huntington’s disease, multiple sclerosis, Parkinson’s disease and schizophrenia using an extensive cohort of well characterised post-mortem CNS tissues. The analysis of whole genome expr

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import numpy as np

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data using Illumina whole genome BeadChip
is_gene_available = True

# 2.1 Data Availability
# For trait: Key 0 contains disease information including "Amyotrophic lateral sclerosis"
trait_row = 0

# For age: Key 2 contains age information
age_row = 2

# For gender: Key 1 contains gender information
gender_row = 1

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait (disease) values to binary format for ALS."""
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary (1 for ALS, 0 for other diseases)
    if "amyotrophic lateral sclerosis" in value.lower():
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age values to continuous format."""
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Try to extract numeric age value
    try:
        # Extract only digits from the string
        if "age at death (in years):" in value.lower():
            age_value = value.lower().replace("age at death (in years):", "").strip()
        else:
            age_value = value
        return float(age_value)
    except:
        return None

def convert_gender(value):
    """Convert gender values to binary format (0 for female, 1 for male)."""
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None

# Helper function to extract feature data (same as would be used in geo_select_clinical_features)
def get_feature_data(clinical_df, row_idx, feature_name, convert_func):
    """Extract and convert a feature from the clinical dataframe."""
    # Get the data for the specified row
    feature_values = clinical_df.iloc[row_idx].tolist()
    
    # Convert the values using the provided conversion function
    converted_values = [convert_func(val) for val in feature_values]
    
    # Create a Series with the converted values
    return pd.Series(converted_values, name=feature_name)

# 3. Save Metadata - Initial Filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait_row is not None)
if trait_row is not None:
    # Ensure the output directory exists
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Create a dataframe from the sample characteristics dictionary
    sample_chars = {
        0: ["disease: Alzheimer's disease", 'disease: Amyotrophic lateral sclerosis', "disease: Huntington's disease", 'disease: Multiple sclerosis', "disease: Parkinson's disease", 'disease: Schizophrenia'],
        1: ['gender: M', 'gender: F'],
        2: ['age at death (in years): 70', 'age at death (in years): 73', 'age at death (in years): 59', 'age at death (in years): 40', 'age at death (in years): 47', 'age at death (in years): 82', 'age at death (in years): 86', 'age at death (in years): 93', 'age at death (in years): 72', 'age at death (in years): 85', 'age at death (in years): 80', 'age at death (in years): 79', 'age at death (in years): 76', 'age at death (in years): 77', 'age at death (in years): 55', 'age at death (in years): 43', 'age at death (in years): 39', 'age at death (in years): 67', 'age at death (in years): 84', 'age at death (in years): 54', 'age at death (in years): 74', 'age at death (in years): 69', 'age at death (in years): 64', 'age at death (in years): 60', 'age at death (in years): 68', 'age at death (in years): 18', 'age at death (in years): 57', 'age at death (in years): 46', 'age at death (in years): 50', 'age at death (in years): 53']
    }
    
    # Determine the max length of lists to create our dataframe
    max_length = max(len(values) for values in sample_chars.values())
    
    # Fill shorter lists with NaN to ensure consistent lengths
    for key in sample_chars:
        if len(sample_chars[key]) < max_length:
            sample_chars[key] = sample_chars[key] + [np.nan] * (max_length - len(sample_chars[key]))
    
    # Create the clinical dataframe from the sample characteristics
    clinical_data = pd.DataFrame(sample_chars)
    
    # Extract clinical features using similar steps as geo_select_clinical_features
    feature_list = []
    
    # Add trait data
    trait_data = get_feature_data(clinical_data, trait_row, trait, convert_trait)
    feature_list.append(trait_data)
    
    # Add age data if available
    if age_row is not None:
        age_data = get_feature_data(clinical_data, age_row, 'Age', convert_age)
        feature_list.append(age_data)
    
    # Add gender data if available
    if gender_row is not None:
        gender_data = get_feature_data(clinical_data, gender_row, 'Gender', convert_gender)
        feature_list.append(gender_data)
    
    # Combine all features into a single dataframe
    selected_clinical_df = pd.concat(feature_list, axis=1)
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the selected clinical features to a CSV file
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'Amyotrophic_Lateral_Sclerosis': [0, 0, 0], 'Age': [nan, nan, 59.0], 'Gender': [nan, 0.0, nan]}
Clinical data saved to ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/clinical_data/GSE26927.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['ILMN_10000', 'ILMN_10001', 'ILMN_10002', 'ILMN_10004', 'ILMN_10005',
       'ILMN_10006', 'ILMN_10009', 'ILMN_1001', 'ILMN_10010', 'ILMN_10011',
       'ILMN_10012', 'ILMN_10013', 'ILMN_10014', 'ILMN_10016', 'ILMN_1002',
       'ILMN_10020', 'ILMN_10021', 'ILMN_10022', 'ILMN_10023', 'ILMN_10024'],
      dtype='object', name='ID')

Gene data dimensions: 20589 genes × 118 samples


### Step 4: Gene Identifier Review

In [5]:
# These gene identifiers start with "ILMN_" which indicates they are Illumina microarray probe IDs
# These are not standard human gene symbols and would need to be mapped to official gene symbols
# ILMN_ prefixes are used by Illumina BeadArray platforms and require conversion

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_10000', 'ILMN_10001', 'ILMN_10002', 'ILMN_10004', 'ILMN_10005'], 'GB_ACC': ['NM_007112.3', 'NM_018976.3', 'NM_175569.1', 'NM_001954.3', 'NM_031966.2'], 'SYMBOL': ['THBS3', 'SLC38A2', 'XG', 'DDR1', 'CCNB1'], 'DEFINITION': ['Homo sapiens thrombospondin 3 (THBS3), mRNA.', 'Homo sapiens solute carrier family 38, member 2 (SLC38A2), mRNA.', 'Homo sapiens Xg blood group (XG), mRNA.', 'Homo sapiens discoidin domain receptor family, member 1 (DDR1), transcript variant 2, mRNA.', 'Homo sapiens cyclin B1 (CCNB1), mRNA.'], 'ONTOLOGY': ['cell-matrix adhesion [goid 7160] [pmid 8468055] [evidence TAS]; cell motility [goid 6928] [evidence NR ]; calcium ion binding [goid 5509] [pmid 8288588] [evidence TAS]; structural molecule activity [goid 5198] [evidence IEA]; protein binding [goid 5515] [evidence IEA]; heparin binding [goid 8201] [evidence NR ]; extracellular matrix (sensu Metazoa) [goid 5578] [evidence NR ]', 'transport [goid 6810] [evidence IEA]; amino acid

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns contain gene identifiers and gene symbols
# Based on the gene annotation preview, we can see:
# - 'ID' column contains Illumina probe IDs (ILMN_*) matching the gene expression data index
# - 'SYMBOL' column contains the human gene symbols we need to map to

# 2. Get gene mapping dataframe by extracting ID and SYMBOL columns
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'SYMBOL')

# Print a preview of the mapping dataframe
print("Gene mapping preview:")
print(preview_df(mapping_df))

# 3. Convert probe-level measurements to gene expression data by applying the gene mapping
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print information about the gene expression data after mapping
print(f"\nGene expression data after mapping: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("\nFirst 5 gene symbols:")
print(gene_data.index[:5])

# Create the output directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the gene expression data to a CSV file
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping preview:
{'ID': ['ILMN_10000', 'ILMN_10001', 'ILMN_10002', 'ILMN_10004', 'ILMN_10005'], 'Gene': ['THBS3', 'SLC38A2', 'XG', 'DDR1', 'CCNB1']}



Gene expression data after mapping: 17613 genes × 118 samples

First 5 gene symbols:
Index(['A1BG', 'A2BP1', 'A2M', 'A2ML1', 'A4GALT'], dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE26927.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the index of gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 gene symbols after normalization: {normalized_gene_data.index[:5]}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if clinical data was properly loaded
# First, reload the clinical_data to make sure we're using the original data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Print the sample IDs to understand the data structure
print("Sample IDs in clinical data:")
print(clinical_data.columns[:5], "...")  # Show first 5 sample IDs

# Print the sample IDs in gene expression data
print("Sample IDs in gene expression data:")
print(normalized_gene_data.columns[:5], "...")  # Show first 5 sample IDs

# Extract clinical features using the actual sample IDs
is_trait_available = trait_row is not None
linked_data = None

if is_trait_available:
    # Extract clinical features with proper sample IDs
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    print(f"Clinical data preview: {preview_df(selected_clinical_df, n=3)}")
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    # Make sure both dataframes have compatible indices/columns
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before handling missing values: {linked_data.shape}")
    
    if linked_data.shape[0] == 0:
        print("WARNING: No samples matched between clinical and genetic data!")
        # Create a sample dataset for demonstration
        print("Using gene data with artificial trait values for demonstration")
        is_trait_available = False
        is_biased = True
        linked_data = pd.DataFrame(index=normalized_gene_data.columns)
        linked_data[trait] = 1  # Placeholder
    else:
        # 3. Handle missing values
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        # 4. Determine if trait and demographic features are biased
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
else:
    print("Trait data was determined to be unavailable in previous steps.")
    is_biased = True  # Set to True since we can't evaluate without trait data
    linked_data = pd.DataFrame(index=normalized_gene_data.columns)
    linked_data[trait] = 1  # Add a placeholder trait column
    print(f"Using placeholder data due to missing trait information, shape: {linked_data.shape}")

# 5. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from multiple sclerosis patients, but there were issues linking clinical and genetic data."
)

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associational studies.")

Gene data shape after normalization: (16595, 118)
First 5 gene symbols after normalization: Index(['A1BG', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT'], dtype='object', name='Gene')


Normalized gene data saved to ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE26927.csv
Sample IDs in clinical data:
Index(['!Sample_geo_accession', 'GSM663008', 'GSM663009', 'GSM663010',
       'GSM663011'],
      dtype='object') ...
Sample IDs in gene expression data:
Index(['GSM663008', 'GSM663009', 'GSM663010', 'GSM663011', 'GSM663012'], dtype='object') ...
Clinical data shape: (3, 118)
Clinical data preview: {'GSM663008': [0.0, 70.0, 1.0], 'GSM663009': [0.0, 73.0, 0.0], 'GSM663010': [0.0, 73.0, 0.0], 'GSM663011': [0.0, 59.0, 1.0], 'GSM663012': [0.0, 40.0, 1.0], 'GSM663013': [0.0, 47.0, 0.0], 'GSM663014': [0.0, 47.0, 0.0], 'GSM663015': [0.0, 82.0, 0.0], 'GSM663016': [0.0, 86.0, 0.0], 'GSM663017': [0.0, 82.0, 0.0], 'GSM663018': [0.0, 93.0, 1.0], 'GSM663019': [0.0, 82.0, 0.0], 'GSM663020': [0.0, 72.0, 1.0], 'GSM663021': [0.0, 85.0, 1.0], 'GSM663022': [0.0, 80.0, 1.0], 'GSM663023': [0.0, 79.0, 1.0], 'GSM663024': [0.0, 76.0, 1.0], 'GSM663025': [0.0, 77.0, 0.0], 'GSM6

Data shape after handling missing values: (118, 16598)
For the feature 'Amyotrophic_Lateral_Sclerosis', the least common label is '1.0' with 20 occurrences. This represents 16.95% of the dataset.
The distribution of the feature 'Amyotrophic_Lateral_Sclerosis' in this dataset is fine.

Quartiles for 'Age':
  25%: 53.0
  50% (Median): 64.0
  75%: 77.0
Min: 18.0
Max: 104.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 42 occurrences. This represents 35.59% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Data shape after removing biased features: (118, 16598)


Linked data saved to ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/GSE26927.csv
